<a href="https://colab.research.google.com/github/hardiksiloiya/GANs/blob/main/Variational_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import keras
from keras.layers import Input,Conv2DTranspose,Conv2D,BatchNormalization,LeakyReLU,Dropout,Flatten,Lambda,Dense,Reshape,Activation
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import mnist
import numpy as np





In [104]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test=x_test.astype(np.float32)
y_test=y_test.astype(np.float32)

In [105]:
def func(inp):
  u,log_var=inp
  epsilon = K.random_normal(shape=K.shape(u), mean=0., stddev=1.)
  return u+K.exp(log_var/2)*epsilon

encoder_input=Input((28,28,1))
x=encoder_input
x=Conv2D(filters=32,kernel_size=3,strides=1,padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.25)(x)
x=LeakyReLU()(x)
x=Conv2D(filters=32,kernel_size=3,strides=2,padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.25)(x)
x=LeakyReLU()(x)
x=Conv2D(filters=16,kernel_size=3,strides=2,padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.25)(x)
x=LeakyReLU()(x)
x=Conv2D(filters=16,kernel_size=3,strides=1,padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.25)(x)
x=LeakyReLU()(x)
x=Flatten()(x)
u=Dense(2)(x)
log_var=Dense(2)(x)

encoder_out=Lambda(func)([u,log_var])    #sampling layer
en_model=Model(encoder_input,encoder_out)
#en_model.summary()


In [106]:
decoder_input=Input(2)
y=Dense(1568)(decoder_input)
y=Reshape((7,7,32))(y)
y=Conv2DTranspose(filters=10,kernel_size=3,strides=2,padding='same')(y)
y=BatchNormalization()(y)
y=Dropout(rate=0.25)(y)
y=LeakyReLU()(y)
y=Conv2DTranspose(filters=10,kernel_size=3,strides=2,padding='same')(y)
y=BatchNormalization()(y)
y=Dropout(rate=0.25)(y)
y=LeakyReLU()(y)
y=Conv2DTranspose(filters=10,kernel_size=3,strides=1,padding='same')(y)
y=BatchNormalization()(y)
y=Dropout(rate=0.25)(y)
y=LeakyReLU()(y)
y=Conv2DTranspose(filters=1,kernel_size=3,strides=1,padding='same')(y)
y=BatchNormalization()(y)
y=Dropout(rate=0.25)(y)
y=Activation('sigmoid')(y)
decoder_out=y
de_model=Model(decoder_input,decoder_out)
#de_model.summary()

In [107]:
model=Model(encoder_input,de_model(encoder_out))
model.summary()

Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 28, 28, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_72 (BatchNo (None, 28, 28, 32)   128         conv2d_45[0][0]                  
__________________________________________________________________________________________________
dropout_71 (Dropout)            (None, 28, 28, 32)   0           batch_normalization_72[0][0]     
___________________________________________________________________________________________

In [108]:
optimizer=Adam(lr=0.0005)
def squareloss(a,b):
  return 1000*K.mean(K.square(a-b), axis = [1,2,3])
def kl_div(a,b):
  kl_loss = -0.5 * K.sum(1+log_var-K.square(u)-K.exp(log_var),axis = 1)
  return kl_loss
def loss(a,b):
  return squareloss(a,b)+kl_div(a,b)
model.compile(optimizer=optimizer,loss=loss,metrics=[squareloss, kl_div])

In [ ]:
model.fit(x=x_train,y=x_train,batch_size=32,shuffle=True,epochs=10)

Train on 60000 samples
Epoch 1/10
29568/60000 [=============>................] - ETA: 1:45 - loss: 7220542.1521 - squareloss: 7220526.5000 - kl_div: 18.0451